In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

tf.random.set_seed(0)

2025-03-03 15:53:44.511422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741013624.556145 1271703 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741013624.569841 1271703 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-03 15:53:44.614942: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
sampling_rate = "1s"
prediction_column = "mid_price_variation_class"
batch_size = 16
look_back = 32

In [3]:
from deeplob_preprocessing import process_and_combine_data

start_date = "2024-10-01"
end_date = "2024-10-09"

all_data = process_and_combine_data(start_date, end_date, data_folder="/Data/harold.ngoupeyou/data_challenge/Stock-Prediction-Deep-Leaning/AAPL_data/DB_MBP_10/AAPL", sampling_rate=sampling_rate)

print(all_data.columns)

all_data.head()

Index(['bid_px_00', 'ask_px_00', 'bid_sz_00', 'ask_sz_00', 'bid_px_01',
       'ask_px_01', 'bid_sz_01', 'ask_sz_01', 'bid_px_02', 'ask_px_02',
       'bid_sz_02', 'ask_sz_02', 'bid_px_03', 'ask_px_03', 'bid_sz_03',
       'ask_sz_03', 'bid_px_04', 'ask_px_04', 'bid_sz_04', 'ask_sz_04',
       'bid_px_05', 'ask_px_05', 'bid_sz_05', 'ask_sz_05', 'bid_px_06',
       'ask_px_06', 'bid_sz_06', 'ask_sz_06', 'bid_px_07', 'ask_px_07',
       'bid_sz_07', 'ask_sz_07', 'bid_px_08', 'ask_px_08', 'bid_sz_08',
       'ask_sz_08', 'bid_px_09', 'ask_px_09', 'bid_sz_09', 'ask_sz_09',
       'mid_price_variation_class'],
      dtype='object')


,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_px_01,ask_px_01,bid_sz_01,ask_sz_01,bid_px_02,ask_px_02,...,ask_sz_07,bid_px_08,ask_px_08,bid_sz_08,ask_sz_08,bid_px_09,ask_px_09,bid_sz_09,ask_sz_09,mid_price_variation_class
ts_recv,,,,,,,,,,,,,,,,,,,,,
2024-10-01 09:30:01-04:00,229.00,229.05,911.0,200.0,228.99,229.06,101.0,72.0,228.97,229.07,...,751.0,228.88,229.13,11.0,203.0,228.87,229.14,1.0,53.0,0.0
2024-10-01 09:30:02-04:00,229.13,229.17,50.0,748.0,229.12,229.19,150.0,122.0,229.11,229.20,...,147.0,229.00,229.26,7721.0,72.0,228.99,229.27,51.0,100.0,2.0
2024-10-01 09:30:03-04:00,229.12,229.17,137.0,150.0,229.10,229.18,125.0,72.0,229.09,229.19,...,50.0,229.01,229.30,3950.0,150.0,229.00,229.31,7696.0,150.0,0.0
2024-10-01 09:30:04-04:00,229.12,229.15,50.0,50.0,229.11,229.16,50.0,331.0,229.09,229.17,...,51.0,229.00,229.23,7696.0,101.0,228.99,229.25,101.0,100.0,0.0
2024-10-01 09:30:05-04:00,229.13,229.16,50.0,50.0,229.12,229.17,75.0,50.0,229.11,229.18,...,101.0,229.05,229.24,100.0,82.0,229.03,229.25,50.0,100.0,2.0


In [4]:
correct_order = []
num_levels = 10 

for i in range(num_levels):
    correct_order.append(f"bid_px_0{i}")
    correct_order.append(f"bid_sz_0{i}")
    correct_order.append(f"ask_px_0{i}")
    correct_order.append(f"ask_sz_0{i}")

correct_order.append("mid_price_variation_class")
df = all_data[correct_order]
print(df.columns)

df.head()


Index(['bid_px_00', 'bid_sz_00', 'ask_px_00', 'ask_sz_00', 'bid_px_01',
       'bid_sz_01', 'ask_px_01', 'ask_sz_01', 'bid_px_02', 'bid_sz_02',
       'ask_px_02', 'ask_sz_02', 'bid_px_03', 'bid_sz_03', 'ask_px_03',
       'ask_sz_03', 'bid_px_04', 'bid_sz_04', 'ask_px_04', 'ask_sz_04',
       'bid_px_05', 'bid_sz_05', 'ask_px_05', 'ask_sz_05', 'bid_px_06',
       'bid_sz_06', 'ask_px_06', 'ask_sz_06', 'bid_px_07', 'bid_sz_07',
       'ask_px_07', 'ask_sz_07', 'bid_px_08', 'bid_sz_08', 'ask_px_08',
       'ask_sz_08', 'bid_px_09', 'bid_sz_09', 'ask_px_09', 'ask_sz_09',
       'mid_price_variation_class'],
      dtype='object')


,bid_px_00,bid_sz_00,ask_px_00,ask_sz_00,bid_px_01,bid_sz_01,ask_px_01,ask_sz_01,bid_px_02,bid_sz_02,...,ask_sz_07,bid_px_08,bid_sz_08,ask_px_08,ask_sz_08,bid_px_09,bid_sz_09,ask_px_09,ask_sz_09,mid_price_variation_class
ts_recv,,,,,,,,,,,,,,,,,,,,,
2024-10-01 09:30:01-04:00,229.00,911.0,229.05,200.0,228.99,101.0,229.06,72.0,228.97,100.0,...,751.0,228.88,11.0,229.13,203.0,228.87,1.0,229.14,53.0,0.0
2024-10-01 09:30:02-04:00,229.13,50.0,229.17,748.0,229.12,150.0,229.19,122.0,229.11,200.0,...,147.0,229.00,7721.0,229.26,72.0,228.99,51.0,229.27,100.0,2.0
2024-10-01 09:30:03-04:00,229.12,137.0,229.17,150.0,229.10,125.0,229.18,72.0,229.09,50.0,...,50.0,229.01,3950.0,229.30,150.0,229.00,7696.0,229.31,150.0,0.0
2024-10-01 09:30:04-04:00,229.12,50.0,229.15,50.0,229.11,50.0,229.16,331.0,229.09,50.0,...,51.0,229.00,7696.0,229.23,101.0,228.99,101.0,229.25,100.0,0.0
2024-10-01 09:30:05-04:00,229.13,50.0,229.16,50.0,229.12,75.0,229.17,50.0,229.11,100.0,...,101.0,229.05,100.0,229.24,82.0,229.03,50.0,229.25,100.0,2.0


In [5]:
df.mid_price_variation_class.value_counts()

mid_price_variation_class
1.0    64272
2.0    50464
0.0    48782
Name: count, dtype: int64

In [6]:
import pandas as pd
import numpy as np

def compute_normalization_params(df, feature_cols):
    """
    Z-score
    """
    normalization_params = {}
    for col in feature_cols:
        mean = df[col].mean()
        std = df[col].std()
        std = std if std > 1e-8 else 1
        normalization_params[col] = {"mean": mean, "std": std}
    return normalization_params


def normalize_df(df, normalization_params):
    
    df_normalized = df.copy()
    for col, params in normalization_params.items():
        df_normalized[col] = (df[col] - params["mean"]) / params["std"]
    return df_normalized


train_size = int(0.7 * len(df))
val_size = int(0.2 * len(df))

train_df = df.iloc[:train_size]
val_df = df.iloc[train_size:train_size + val_size]
test_df = df.iloc[train_size + val_size:]

price_cols = [col for col in df.columns if col.startswith("bid_px_") or col.startswith("ask_px_")]
volume_cols = [col for col in df.columns if col.startswith("bid_sz_") or col.startswith("ask_sz_")]

normalization_params_prices = compute_normalization_params(train_df, price_cols)
normalization_params_volumes = compute_normalization_params(train_df, volume_cols)

train_df = normalize_df(train_df, {**normalization_params_prices, **normalization_params_volumes})
val_df = normalize_df(val_df, {**normalization_params_prices, **normalization_params_volumes})
test_df = normalize_df(test_df, {**normalization_params_prices, **normalization_params_volumes})

print(f"Train: {train_df.shape}, Val: {val_df.shape}, Test: {test_df.shape}")
train_df.head()


Train: (114462, 41), Val: (32703, 41), Test: (16353, 41)


,bid_px_00,bid_sz_00,ask_px_00,ask_sz_00,bid_px_01,bid_sz_01,ask_px_01,ask_sz_01,bid_px_02,bid_sz_02,...,ask_sz_07,bid_px_08,bid_sz_08,ask_px_08,ask_sz_08,bid_px_09,bid_sz_09,ask_px_09,ask_sz_09,mid_price_variation_class
ts_recv,,,,,,,,,,,,,,,,,,,,,
2024-10-01 09:30:01-04:00,3.596304,1.535431,3.627262,0.026222,3.596494,-0.388046,3.627139,-0.473728,3.587128,-0.370876,...,0.431305,3.559017,-0.426750,3.627036,-0.326265,3.559040,-0.386752,3.627011,-0.431851,0.0
2024-10-01 09:30:02-04:00,3.718985,-0.382782,3.740472,1.765909,3.719178,-0.269077,3.749782,-0.342860,3.719251,-0.150917,...,-0.411800,3.672271,7.733883,3.749677,-0.499785,3.672294,-0.339109,3.749651,-0.381566,2.0
2024-10-01 09:30:03-04:00,3.709548,-0.188955,3.740472,-0.132508,3.700303,-0.329776,3.740348,-0.473728,3.700376,-0.480855,...,-0.547199,3.681708,3.742476,3.787413,-0.396468,3.681731,6.945558,3.787387,-0.328072,0.0
2024-10-01 09:30:04-04:00,3.709548,-0.382782,3.721603,-0.449970,3.709740,-0.511871,3.721480,0.204169,3.700376,-0.480855,...,-0.545803,3.672271,7.707422,3.721375,-0.461372,3.672294,-0.291465,3.730784,-0.381566,0.0
2024-10-01 09:30:05-04:00,3.718985,-0.382782,3.731038,-0.449970,3.719178,-0.451173,3.730914,-0.531310,3.719251,-0.370876,...,-0.476010,3.719459,-0.332548,3.730809,-0.486539,3.710045,-0.340061,3.730784,-0.381566,2.0


In [7]:
from dl_series_generators import TimeSeriesDeepLOBGenerator

features = [
    "bid_px_00", "bid_sz_00", "ask_px_00", "ask_sz_00",
    "bid_px_01", "bid_sz_01", "ask_px_01", "ask_sz_01",
    "bid_px_02", "bid_sz_02", "ask_px_02", "ask_sz_02",
    "bid_px_03", "bid_sz_03", "ask_px_03", "ask_sz_03",
    "bid_px_04", "bid_sz_04", "ask_px_04", "ask_sz_04",
    "bid_px_05", "bid_sz_05", "ask_px_05", "ask_sz_05",
    "bid_px_06", "bid_sz_06", "ask_px_06", "ask_sz_06",
    "bid_px_07", "bid_sz_07", "ask_px_07", "ask_sz_07",
    "bid_px_08", "bid_sz_08", "ask_px_08", "ask_sz_08",
    "bid_px_09", "bid_sz_09", "ask_px_09", "ask_sz_09"
]


train_gen = TimeSeriesDeepLOBGenerator(
    data=train_df, target=prediction_column, features=features,
    look_back=look_back, batch_size=batch_size
)
val_gen = TimeSeriesDeepLOBGenerator(
    data=val_df, target=prediction_column, features=features,
    look_back=look_back, batch_size=batch_size
)
test_gen = TimeSeriesDeepLOBGenerator(
    data=test_df, target=prediction_column, features=features,
    look_back=look_back, batch_size=batch_size
)

In [8]:
X_train = np.concatenate([train_gen[i][0] for i in range(len(train_gen))], axis=0)
y_train = np.concatenate([train_gen[i][1] for i in range(len(train_gen))], axis=0)
X_val = np.concatenate([val_gen[i][0] for i in range(len(val_gen))], axis=0)
y_val = np.concatenate([val_gen[i][1] for i in range(len(val_gen))], axis=0)
print(f"X_train shape: {X_train.shape}")  # (total_samples, look_back, num_features)
print(f"y_train shape: {y_train.shape}")  # (total_samples,)


X_train shape: (114430, 32, 40)
y_train shape: (114430,)


In [9]:
import torch
y_train_tensor = torch.tensor(y_train, dtype=torch.long) 

print(torch.bincount(y_train_tensor))

tensor([35892, 41716, 36822])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import time


class DeepLOBModel(nn.Module):
    def __init__(self, lookback_timestep, feature_num, conv_filter_num, inception_num, LSTM_num, leaky_relu_alpha):
       
        super(DeepLOBModel, self).__init__()
        self.leaky_relu_alpha = leaky_relu_alpha

        # ---  Convolution block 1 ---
        # input shape : (batch, 1, lookback_timestep, feature_num)
        self.conv1_1 = nn.Conv2d(1, conv_filter_num, kernel_size=(1,2), stride=(1,2))
        self.conv1_2 = nn.Conv2d(conv_filter_num, conv_filter_num, kernel_size=(4,1), padding='same')
        self.conv1_3 = nn.Conv2d(conv_filter_num, conv_filter_num, kernel_size=(4,1), padding='same')

        # --- Convolution block 2 ---
        self.conv2_1 = nn.Conv2d(conv_filter_num, conv_filter_num, kernel_size=(1,2), stride=(1,2))
        self.conv2_2 = nn.Conv2d(conv_filter_num, conv_filter_num, kernel_size=(4,1), padding='same')
        self.conv2_3 = nn.Conv2d(conv_filter_num, conv_filter_num, kernel_size=(4,1), padding='same')

        # --- Convolution block 3 ---
        self.conv3_1 = nn.Conv2d(conv_filter_num, conv_filter_num, kernel_size=(1,10), padding=0)  # padding='valid'
        self.conv3_2 = nn.Conv2d(conv_filter_num, conv_filter_num, kernel_size=(4,1), padding='same')
        self.conv3_3 = nn.Conv2d(conv_filter_num, conv_filter_num, kernel_size=(4,1), padding='same')

        # ---  Inception Module ---
        # Path 1
        self.inception1_1 = nn.Conv2d(conv_filter_num, inception_num, kernel_size=(1,1), padding='same')
        self.inception1_2 = nn.Conv2d(inception_num, inception_num, kernel_size=(3,1), padding='same')
        # Path 2
        self.inception2_1 = nn.Conv2d(conv_filter_num, inception_num, kernel_size=(1,1), padding='same')
        self.inception2_2 = nn.Conv2d(inception_num, inception_num, kernel_size=(5,1), padding='same')
        # Path 3
        self.inception3_pool = nn.MaxPool2d(kernel_size=(3,1), stride=(1,1), padding=(1,0))
        self.inception3_1 = nn.Conv2d(conv_filter_num, inception_num, kernel_size=(1,1), padding='same')

        # --- LSTM layer ---
        self.lstm = nn.LSTM(input_size=3*inception_num, hidden_size=LSTM_num, batch_first=True)

        # ---  Fully Connected layer ---
        self.fc = nn.Linear(LSTM_num, 3)

    def forward(self, x):
        # x: (batch, lookback_timestep, feature_num)
        x = x.unsqueeze(1)

        x = F.leaky_relu(self.conv1_1(x), negative_slope=self.leaky_relu_alpha)
        x = F.leaky_relu(self.conv1_2(x), negative_slope=self.leaky_relu_alpha)
        x = F.leaky_relu(self.conv1_3(x), negative_slope=self.leaky_relu_alpha)

        x = F.leaky_relu(self.conv2_1(x), negative_slope=self.leaky_relu_alpha)
        x = F.leaky_relu(self.conv2_2(x), negative_slope=self.leaky_relu_alpha)
        x = F.leaky_relu(self.conv2_3(x), negative_slope=self.leaky_relu_alpha)

        x = F.leaky_relu(self.conv3_1(x), negative_slope=self.leaky_relu_alpha)
        x = F.leaky_relu(self.conv3_2(x), negative_slope=self.leaky_relu_alpha)
        x = F.leaky_relu(self.conv3_3(x), negative_slope=self.leaky_relu_alpha)


        i1 = F.leaky_relu(self.inception1_1(x), negative_slope=self.leaky_relu_alpha)
        i1 = F.leaky_relu(self.inception1_2(i1), negative_slope=self.leaky_relu_alpha)

        i2 = F.leaky_relu(self.inception2_1(x), negative_slope=self.leaky_relu_alpha)
        i2 = F.leaky_relu(self.inception2_2(i2), negative_slope=self.leaky_relu_alpha)

        i3 = self.inception3_pool(x)
        i3 = F.leaky_relu(self.inception3_1(i3), negative_slope=self.leaky_relu_alpha)

        x_incep = torch.cat([i1, i2, i3], dim=1)

        batch_size, channels, H, W = x_incep.size()
        x_incep = x_incep.view(batch_size, H, channels * W)

        lstm_out, _ = self.lstm(x_incep)
        lstm_last = lstm_out[:, -1, :]

        out = self.fc(lstm_last)
        return out




X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)


train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


##### Hparams #####
feature_num = 40  
conv_filter_num = 8 
inception_num = 16 
LSTM_num = 32 
leaky_relu_alpha = 0.01

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = DeepLOBModel(look_back, feature_num, conv_filter_num, inception_num, LSTM_num, leaky_relu_alpha)
model.to(device)

# Normalized loss according to the classes distribution
class_counts = torch.tensor([35892, 41716, 36822], dtype=torch.float32)
class_weights = 1.0 / class_counts
class_weights = class_weights / class_weights.sum() 
print(f"Class weights: {class_weights}")

criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))  

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4,)


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

##################### Training #######################
num_epoch = 100  
patience = 20    # early stopping
best_val_acc = 0
epochs_no_improve = 0

for epoch in range(num_epoch):
    model.train()
    epoch_loss = 0.0
    correct = 0
    total = 0
    start_time = time.time()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs) 

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        
       # if (batch_idx + 1) % 750 == 0:
          #  print(f"Epoch {epoch+1} Batch {batch_idx+1}/{len(train_loader)}: Loss = {loss.item():.4f}")
    
    epoch_loss /= total
    train_acc = correct / total
    
    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += targets.size(0)
            val_correct += (predicted == targets).sum().item()
    
    val_loss /= val_total
    val_acc = val_correct / val_total
    
    elapsed = time.time() - start_time
    print(f"\nEpoch {epoch+1}: Train Loss = {epoch_loss:.4f}, Train Acc = {train_acc:.4f}, " \
          f"Val Loss = {val_loss:.4f}, Val Acc = {val_acc:.4f}, Time = {elapsed:.2f}s")
    
    scheduler.step(val_loss)
    
    # Early stopping check
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        epochs_no_improve = 0
        best_model_state = model.state_dict()
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break


cuda
Class weights: tensor([0.3527, 0.3035, 0.3438])


/Data/harold.ngoupeyou/cheese_challenge/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/Data/harold.ngoupeyou/cheese_challenge/lib/python3.12/site-packages/torch/nn/modules/conv.py:549: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1036.)
  return F.conv2d(



Epoch 1: Train Loss = 1.0688, Train Acc = 0.4108, Val Loss = 1.0636, Val Acc = 0.4389, Time = 143.62s

Epoch 2: Train Loss = 1.0549, Train Acc = 0.4326, Val Loss = 1.0548, Val Acc = 0.4473, Time = 147.43s

Epoch 3: Train Loss = 1.0503, Train Acc = 0.4367, Val Loss = 1.0486, Val Acc = 0.4558, Time = 137.20s

Epoch 4: Train Loss = 1.0479, Train Acc = 0.4378, Val Loss = 1.0489, Val Acc = 0.4563, Time = 136.09s

Epoch 5: Train Loss = 1.0462, Train Acc = 0.4395, Val Loss = 1.0498, Val Acc = 0.4499, Time = 135.07s

Epoch 6: Train Loss = 1.0455, Train Acc = 0.4404, Val Loss = 1.0473, Val Acc = 0.4596, Time = 141.92s

Epoch 7: Train Loss = 1.0446, Train Acc = 0.4413, Val Loss = 1.0474, Val Acc = 0.4547, Time = 135.81s

Epoch 8: Train Loss = 1.0438, Train Acc = 0.4429, Val Loss = 1.0534, Val Acc = 0.4447, Time = 133.30s

Epoch 9: Train Loss = 1.0432, Train Acc = 0.4429, Val Loss = 1.0490, Val Acc = 0.4577, Time = 130.64s

Epoch 10: Train Loss = 1.0426, Train Acc = 0.4431, Val Loss = 1.0483, Va